In [1]:
import logging
from pathlib import Path

logging.basicConfig()

survey_name = "P738"
om_working_dir = "C:/Luke/PhD/oasis montaj/ArbSR"
nominal_line_spacing = 250  # m
decimation_factors = [1, 2, 3, 4]
xmin, ymin, xmax, ymax = (
    356800,
    6124400,
    504350,
    6350900,
)  # W, S, E, N / xmin, ymin, xmax, ymax


def create_sel_file(
    in_dir="",
    stats_file_name = "stats.txt",
    out_prefix="decimated",
    data_line_type=2,
    tie_line_type=4,
    decimation_factors=[1],
):
    """Prepare a selection file for Oasis Montaj, based on all available lines

    By default, only tie lines "type 4" are dropped - specify a list of
    decimation factors other than the default in order to remove lines.

    After using the selection tool on your line database, run dellines.gx on
    selected lines.

    LSgeo 2021
    """

    line_meta = Path(in_dir) / stats_file_name
    if not line_meta.exists():
        raise FileNotFoundError("Export lineType channel report from OM first!")

    # Get the min, max, average section of statistics report
    tmp = line_meta.read_text().split(
        "------------   -------        -------        ----           --------       ----------- ------------"
    )[-1]

    all_lines = []
    for line in tmp.split("\n"):  # Split by line
        if ":" in line:  # Only care about line number rows, not blank or header
            for i in line.split(","):
                line_name = i.split("      ")[0]
                line_type = i.split("              ")[2]
                all_lines.append([f'"{line_name[:-2]}"', int(line_type)])
                if "." in line_type:
                    raise NotImplementedError(
                        "Line probably contains mixed types, mad sorry bro"
                    )

    for df in decimation_factors:
        decimation_counter = 0
        # Save every n lines
        selection_to_delete = []
        logging.info(f"\n\nRunning Decimation Factor {df}")

        for line in all_lines:
            if line[1] == tie_line_type:
                selection_to_delete.append(line[0])
                logging.info(f"Will delete tie Line {line[0]}")
                continue  # Handle tie lines independent of decimation

            if decimation_counter % df != 0:
                selection_to_delete.append(line[0])
                logging.info(f"Will delete Line {decimation_counter}: {line[0]}")

            decimation_counter += 1  # Start at line 0 with this at end of loop

        sel_file = Path(in_dir) / f"{survey_name}_{out_prefix}_{df}.sel"
        sel_file.write_text("\n".join(selection_to_delete))

        print(sel_file.absolute())


create_sel_file(in_dir=om_working_dir, decimation_factors=decimation_factors)


C:\Luke\PhD\oasis montaj\ArbSR\P738_decimated_1.sel
C:\Luke\PhD\oasis montaj\ArbSR\P738_decimated_2.sel
C:\Luke\PhD\oasis montaj\ArbSR\P738_decimated_3.sel
C:\Luke\PhD\oasis montaj\ArbSR\P738_decimated_4.sel


In [4]:
def create_control_file(
    in_dir="",
    out_prefix="decimated",
    decimation_factor=1,
    hr_line_spacing=None,
    along_line_spacing=None,
    cs_fac=5, # Default line spacing:cell size ratio
    cs=None,
    xmin="",
    ymin="",
    xmax="",
    ymax="",
    bclip="",  # n.b. included in rangrid.con, but not OM help.
    chan="",
    zb="",
    zm="",
    logopt="",
    logmin="",
    idsf="",
    bkd="",
    srd="",
    iwt="",
    edgclp="",
    wtslp="",  # n.b. not included in example rangrid.con, but listed in OM help.
    tol="",
    pastol="",
    itrmax="",
    ti="",
    icgr="",
):

    """Create control file for Minimum Curvature Gridding in Oasis Montaj

    Priority is given to specified args, then python defaults, then OM defaults
    See rangrid.con example in this folder for explanation and OM defaults.
    """

    line_spacing = hr_line_spacing * decimation_factor
    cs = cs or line_spacing / cs_fac # between 1/5 and 1/2 is good
    bkd = bkd if bkd != "" else 10 * cs
    wtslp = wtslp if wtslp != "" else line_spacing / cs  
    # If the data has been collected along traverse lines, it is recommended to set the slope (wtslp) as the number of grid points between the traverse lines

    if cs % 1 != 0:
        logging.info(f"Decimation Factor {decimation_factor} will result in non-decimal cell size. This is supported, but may be undesired.")

    con_str = (
        f"{cs}/\n"
        f"{xmin},{ymin},{xmax},{ymax},{bclip}/\n"
        f"{chan},{zb},{zm},{logopt},{logmin}/\n"
        f"{idsf},{bkd},{srd},{iwt},{edgclp},{wtslp}/\n"
        f"{tol},{pastol},{itrmax},{ti},{icgr}/\n"
    )
    out_file = Path(in_dir) / f"{survey_name}_{out_prefix}_{decimation_factor}_{chan}_{cs:2.1f}_rangrid.con"
    out_file.write_text(con_str)

    print(out_file.absolute().as_posix())


for df in decimation_factors:
    create_control_file(
        in_dir=om_working_dir,
        xmin=xmin,
        ymin=ymin,
        xmax=xmax,
        ymax=ymax,
        chan="mag_awagsLevelled",
        decimation_factor=df,
        hr_line_spacing=nominal_line_spacing,
        cs_fac=4,
        tol=1, # Data range ~ -500, 500
        ti=0,
        itrmax=200,
    )


C:/Luke/PhD/oasis montaj/ArbSR/P738_decimated_1_mag_awagsLevelled_62.5_rangrid.con
C:/Luke/PhD/oasis montaj/ArbSR/P738_decimated_2_mag_awagsLevelled_125.0_rangrid.con
C:/Luke/PhD/oasis montaj/ArbSR/P738_decimated_3_mag_awagsLevelled_187.5_rangrid.con
C:/Luke/PhD/oasis montaj/ArbSR/P738_decimated_4_mag_awagsLevelled_250.0_rangrid.con
